# Sales Outreach Agent

In [19]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent

# Load environment variables
load_dotenv(override=True)

True

In [8]:
# Define agent personas with distinct tones

professional_prompt = (
    "You are a professional sales agent at TechLink, "
    "an online education company offering AI-driven, career-focused learning programs. "
    "Your goal is to write serious, persuasive cold outreach emails targeted at professionals and educational institutions."
)

engaging_prompt = (
    "You are a witty, engaging sales agent at TechLink, "
    "an online education company offering AI-powered, upskilling programs for tech and business roles. "
    "Craft humorous, attention-grabbing cold emails that spark curiosity and encourage responses."
)

concise_prompt = (
    "You are a fast-moving, efficient sales agent at TechLink, "
    "an online education platform offering smart, AI-powered learning paths. "
    "Write short, compelling cold emails that quickly highlight value and drive action."
)

# Create three distinct AI agents
professional_agent = Agent(
    name="Professional Sales Agent",
    instructions=professional_prompt,
    model="gpt-4o-mini"
)

engaging_agent = Agent(
    name="Engaging Sales Agent",
    instructions=engaging_prompt,
    model="gpt-4o-mini"
)

concise_agent = Agent(
    name="Concise Sales Agent",
    instructions=concise_prompt,
    model="gpt-4o-mini"
)


In [16]:
# Stream and display email content from an agent
email_result = Runner.run_streamed(engaging_agent, input="Write a cold sales email")
async for event in email_result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Unlock Your Future Faster Than a Cat on a Roomba! 🐱🚀

Hi [First Name],

Ever watched a cat ride a Roomba? It’s mesmerizing, and believe me, that’s exactly how fast your career could be zooming forward with our AI-powered upskilling programs at TechLink!

Think about it: in the time it takes to fill your coffee cup, you could be acquiring skills that’ll have employers wondering if you’re a wizard—except instead of casting spells, you’ll be magic with data and tech!

Why TechLink, you ask? 
- **AI-Powered Learning**: Our programs learn how you learn, so it’s like having a personal trainer for your brain!
- **Flexible Schedules**: Learn at your own pace—coffee breaks included, of course!
- **Real-World Projects**: Tackle projects that make your resume pop like popcorn in the microwave!

Curious to see how we can turn your career trajectory from “meh” to “WOW”? Let’s chat! I promise, it’ll be more fun than a cat in a box!

Looking forward to your reply (preferably before I crave a

In [17]:
import asyncio

# The email topic or objective
email_topic = "Write a cold sales email"

# Run all sales agents concurrently to generate email variations
agent_outputs = await asyncio.gather(
    Runner.run(professional_agent,email_topic),
    Runner.run(engaging_agent,email_topic),
    Runner.run(concise_agent,email_topic),
)

# Extract the final text output from each agent
cold_emails = [response.final_output for response in agent_outputs]

# Display all generated emails
for idx, email in enumerate(cold_emails, start=1):
    print(f"\n--- Email Option {idx} ---\n{email}\n")


--- Email Option 1 ---
Subject: Elevate Your Team’s Skills with AI-Driven Learning

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I represent TechLink, an innovative online education company dedicated to empowering professionals through AI-driven, career-focused learning programs.

In today’s rapidly evolving job market, staying ahead means continuously upgrading skills and knowledge. At TechLink, we offer tailored learning solutions that not only equip your team with the latest industry-relevant skills but also empower them to thrive in an increasingly competitive landscape.

Our programs feature:

- **Personalized Learning Paths**: Adaptive courses designed to meet individual career goals.
- **Real-World Applications**: Hands-on projects and case studies to ensure applicable skills.
- **Expert-Led Instruction**: Insights from industry leaders who share relevant experience and wisdom.

We’ve successfully partnered with organizations like [e

In [20]:
# Create an evaluator agent to choose the most appealing email
selector_prompt = (
    "You are a prospective customer receiving cold outreach emails. "
    "Choose the email you're most likely to respond to, based solely on content. "
    "Don't explain your choice—just return the best email as-is."
)

email_selector_agent = Agent(
    name="Email Selector Agent",
    instructions=selector_prompt,
    model="gpt-4o-mini"
)

# Evaluate all email options and pick the most compelling one
with trace("Evaluate & select best cold email"):
    agent_outputs = await asyncio.gather(
        Runner.run(professional_agent,email_topic),
        Runner.run(engaging_agent,email_topic),
        Runner.run(concise_agent,email_topic),
    )

    # Extract email outputs
    email_variants = [res.final_output for res in agent_outputs]

    # Combine all into a single prompt for the selector agent
    evaluation_prompt = "Evaluate the following cold sales emails:\n\n" + "\n\n---\n\n".join(email_variants)

    # Run selector agent
    selected_response = await Runner.run(email_selector_agent, evaluation_prompt)
    
    print("\nBest Sales Email Selected:\n")
    print(selected_response.final_output)




Best Sales Email Selected:

Subject: Transform Your Team's Skills with AI-Driven Learning

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m reaching out from TechLink, where we specialize in revolutionizing professional development through our AI-driven, career-focused learning programs.

In today’s fast-evolving landscape, staying ahead of the curve is essential. We understand that your team’s growth is a priority and that continuous learning is crucial for maintaining a competitive edge. That’s where TechLink can make a significant impact.

Our programs are tailored to meet the unique needs of professionals and educational institutions, offering:

- **Personalized Learning Paths:** Our AI algorithms assess individual skill levels and recommend targeted courses, ensuring your team engages with the most relevant content.
- **Career-Focused Curriculum:** Each program is designed to align with industry demands, equipping learners with the ski

Now go and check out the trace:

https://platform.openai.com/traces

## Before we start - some setup:


Please visit Sendgrid at: https://sendgrid.com/

(Sendgrid is a Twilio company for sending emails.)

Please set up an account - it's free! (at least, for me, right now).

Once you've created an account, click on:

Settings (left sidebar) >> API Keys >> Create API Key (button on top right)

Copy the key to the clipboard, then add a new line to your .env file:

`SENDGRID_API_KEY=xxxx`

And also, within SendGrid, go to:

Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"  
and verify that your own email address is a real email address, so that SendGrid can send emails for you.
